<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático: Redes Geradoras Adversariais (GANs)

As GANs são um dos modelos generativos mais fascinantes em Machine Learning. Elas foram introduzidas por Ian Goodfellow em 2014.

Uma GAN consiste em duas redes neurais que competem em um jogo de soma zero:
1.  **O Gerador (Generator)**: Tenta criar dados sintéticos (ex: imagens) a partir de um ruído aleatório. Seu objetivo é criar dados tão realistas que sejam indistinguíveis dos dados reais. Pense nele como um **falsificador de arte**.
2.  **O Discriminador (Discriminator)**: Atua como um classificador binário. Ele recebe tanto os dados reais do nosso dataset quanto os dados falsos do gerador e tenta identificar quais são reais e quais são falsos. Pense nele como um **crítico de arte**.

O treinamento funciona da seguinte forma:
-   O Discriminador é treinado para ficar melhor em diferenciar o real do falso.
-   O Gerador é treinado para "enganar" o Discriminador, ou seja, para produzir dados que o Discriminador classifique como reais.

Através dessa competição, o Gerador se torna progressivamente melhor na criação de dados realistas. Neste notebook, vamos construir uma **GAN Convolucional Profunda (DCGAN)** para gerar novas imagens de dígitos manuscritos (MNIST).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

print(f"TensorFlow versão: {tf.__version__}")
print("Bibliotecas importadas com sucesso!")

In [ ]:
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Pré-processamento:
# 1. Adicionar uma dimensão de canal (para imagens em tons de cinza)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')

# 2. Normalizar os pixels para o intervalo [-1, 1].
# A ativação 'tanh' na última camada do gerador funciona bem com esta normalização.
x_train = (x_train - 127.5) / 127.5

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Criando um dataset TensorFlow para um pipeline de dados eficiente
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

print(f"Formato dos dados de treino: {x_train.shape}")
print("Dados preparados e normalizados.")


In [ ]:
"""
O Gerador pega um vetor de ruído aleatório e o transforma em uma imagem 28x28.
Ele usa camadas `Conv2DTranspose` para "desconvoluir" o vetor de ruído até o formato de uma imagem.
"""
def build_generator():
    model = models.Sequential(name='Generator')
    # Entrada: vetor de ruído (ex: 100 dimensões)
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))

    # Camada de upsampling 1: 7x7x128
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Camada de upsampling 2: 14x14x64
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Camada de upsampling 3: 28x28x1 (imagem final)
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

generator = build_generator()
generator.summary()

In [ ]:
"""
O Discriminador é um classificador de imagens convolucional padrão.
Ele pega uma imagem 28x28 e retorna um único valor indicando a probabilidade de a imagem ser real.
"""
def build_discriminator():
    model = models.Sequential(name='Discriminator')
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1)) # Saída única para classificação (real ou falso)

    return model

discriminator = build_discriminator()
discriminator.summary()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # O gerador quer que o discriminador pense que as imagens falsas são reais (rótulo 1)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = optimizers.Adam(1e-4)
discriminator_optimizer = optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# Vamos reutilizar este seed para podermos visualizar a evolução das imagens
seed = tf.random.normal([num_examples_to_generate, noise_dim])

# O @tf.function compila a função em um grafo TensorFlow, o que a torna muito mais rápida.
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.suptitle(f'Imagens geradas na Época {epoch}', y=0.92)
    # plt.savefig('image_at_epoch_{:04d}.png'.format(epoch)) # Para salvar as imagens
    plt.show()


In [ ]:
"""
Este é o loop de treinamento principal. Para cada época, iteramos sobre os batches de dados.
Dentro de cada `train_step`, ambas as redes são treinadas. Periodicamente, geramos e
exibimos imagens usando nosso ruído fixo (`seed`) para ver o progresso do Gerador.
"""
print("Iniciando o treinamento da GAN... (Pode levar vários minutos)")
for epoch in range(EPOCHS):
    for image_batch in train_dataset:
        train_step(image_batch)

    # A cada 5 épocas, mostramos o progresso
    if (epoch + 1) % 5 == 0:
        print(f"Época {epoch + 1} concluída.")
        generate_and_save_images(generator, epoch + 1, seed)

# Gerando a imagem final
print("\nTreinamento concluído! Imagens finais geradas:")
generate_and_save_images(generator, EPOCHS, seed)


In [ ]:
"""
### Conclusão

Neste notebook, construímos e treinamos uma GAN do zero. Pudemos observar como, a partir de puro ruído, o Gerador aprendeu a criar imagens que se assemelham a dígitos manuscritos.

**Pontos importantes:**
-   **Treinamento Adversarial:** O coração da GAN é a competição entre o Gerador e o Discriminador.
-   **Instabilidade:** O treinamento de GANs é notoriamente instável e sensível a hiperparâmetros. Pequenas mudanças podem levar a resultados muito diferentes.
-   **Modo Colapso (Mode Collapse):** Um problema comum onde o Gerador aprende a produzir apenas um ou alguns poucos tipos de saídas que enganam bem o Discriminador, em vez de aprender a diversidade completa dos dados.

As GANs são um campo ativo e fascinante de pesquisa, com arquiteturas muito mais avançadas como StyleGAN, CycleGAN e BigGAN, que são capazes de gerar imagens fotorrealistas de alta resolução.
"""